# EXTRACCIÓN DE DATOS:

Paletas de sombras, pintalabios y colorete en función de la paleta de colores asignadas, siendo estas:\
        -PRIMAVERA\
        -VERANO\
        -OTOÑO\
        -INVIERNO

**IMPORTACIONES**

In [3]:
from selenium import webdriver
import pandas as pd
import numpy as np
from webdriver_manager.chrome import ChromeDriverManager
from joblib import Parallel, delayed
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time

Ahora, vamos a scrappear los productos de maquillaje con mejor rating que se ajusten a las paletas de colores\
de cada estación. Sacaremos nombre, precio y rating.

In [ ]:
PATH=ChromeDriverManager().install()

driver=webdriver.Chrome(PATH)  

*Existen varios productos que podemos recomendar segun la colometría de las personas. En este caso, nos centraremos\
en sombras de ojos, pintalabios, colorete y corrector*\

Por otra parte, debemos tener en cuenta que hay varias tiendas donde podemos encontrar productos de maquillaje.\
Concretamente, en nuestro proyecto, los escogeremos de las siguientes:

1. **PRIMOR**
2. **SEPHORA**

# **1 - EXTRACCIÓN DE DATOS DE PRIMOR DE SOMBRAS, PINTALABIOS Y COLORETE DE PRIMOR**

In [ ]:
URL = 'https://www.primor.eu/#/dfclassic/query=paletas%20sombras%20verano&filter%5Bcategories%5D%5B0%5D=Paletas%20de%20sombras%20de%20ojos%20y%20maquillaje&session_id=e68b0b5ac1828fdb430ff5b492adb6e5&query_name=match_and'
driver.get(URL)

In [ ]:
filas = driver.find_elements(By.CLASS_NAME, 'df-card__content')

In [ ]:
#Paletas de sombras
def extraer(URL):
    driver = webdriver.Chrome(PATH)
    driver.get(URL)
    
    driver.execute_script('window.scrollTo(0, 200);')  # 200px de scroll
    
    filas = driver.find_elements(By.CLASS_NAME, 'df-card__content')
    link = driver.find_elements(By.CLASS_NAME, 'df-card__main')
    tmp = []
    links = []

    for e in filas:
        tmp.append(e.text.split('\n'))
    
    for e in link:
        links.append(e.get_attribute('href'))
    
    cols = ['NOMBRE', 'TIPO', 'PRECIO']
    
    total = pd.DataFrame(tmp, columns = cols)
    
    total['LINKS'] = links
     
    return total

In [ ]:
links_sombras_primor = ['https://www.primor.eu/#/dfclassic/query=paletas%20sombras%20verano&filter%5Bcategories%5D%5B0%5D=Paletas%20de%20sombras%20de%20ojos%20y%20maquillaje&session_id=e68b0b5ac1828fdb430ff5b492adb6e5&query_name=match_and',\
               'https://www.primor.eu/#/dfclassic/query=paletas%20sombras%20oto%C3%B1al&session_id=e68b0b5ac1828fdb430ff5b492adb6e5&query_name=match_and',\
               'https://www.primor.eu/#/dfclassic/query=paletas%20sombras%20invierno&filter%5Bcategories%5D%5B0%5D=Ojos%20lujo&filter%5Bcategories%5D%5B1%5D=Paletas%20de%20sombras%20de%20ojos%20y%20maquillaje&filter%5Bcategories%5D%5B2%5D=Sombras%20de%20ojos%20lujo&session_id=e68b0b5ac1828fdb430ff5b492adb6e5&query_name=match_and',\
               'https://www.primor.eu/#/dfclassic/query=paletas%20sombras%20pastel&filter%5Bcategories%5D%5B0%5D=Paletas%20de%20sombras%20de%20ojos%20y%20maquillaje&filter%5Bcategories%5D%5B1%5D=Sombras%20de%20ojos&session_id=e68b0b5ac1828fdb430ff5b492adb6e5&query_name=match_and']

In [ ]:
productos = Parallel(n_jobs=4, verbose=True)(delayed(extraer)(e) for e in links_sombras_primor)

In [ ]:
sombras_ver_prim = productos[0] 
sombras_oto_prim = productos[1] 
sombras_inv_prim = productos[2] 
sombras_pri_prim = productos[3] 
#productos_totales = pd.concat([productos[0], productos[1], productos[2], productos[3]], axis = 0)

In [ ]:
ad1 = extraer('https://www.primor.eu/#/dfclassic/query=sombras%20krash%20rosas&filter%5Bcategories%5D%5B0%5D=Paletas%20de%20sombras%20de%20ojos%20y%20maquillaje&filter%5Bcategories%5D%5B1%5D=Sombras%20de%20ojos&filter%5Bbrand%5D%5B0%5D=Krash%20Kosmetics&session_id=e68b0b5ac1828fdb430ff5b492adb6e5&query_name=match_and')
ad2 = extraer('https://www.primor.eu/#/dfclassic/query=sombras%20lilas&filter%5Bcategories%5D%5B0%5D=Maquillaje%20de%20ojos&session_id=e68b0b5ac1828fdb430ff5b492adb6e5&query_name=match_and')
ad3 = extraer('https://www.primor.eu/#/dfclassic/query=sombras%20negras&filter%5Bcategories%5D%5B0%5D=Maquillaje%20de%20ojos&session_id=e68b0b5ac1828fdb430ff5b492adb6e5&query_name=match_and')
ad4 = extraer('https://www.primor.eu/#/dfclassic/query=sombras%20naranjas&session_id=e68b0b5ac1828fdb430ff5b492adb6e5&query_name=match_and')

In [ ]:
#Limpiamos los índices y añadimos lo que hemos sacado más

#PRIMAVERA
sombras_pri_prim = pd.concat([sombras_pri_prim, ad1], axis = 0)
sombras_pri_prim.drop_duplicates(inplace = True)
sombras_pri_prim['ID'] = [e for e in range(len(sombras_pri_prim))]
sombras_pri_prim.set_index('ID', drop = True, inplace = True)

#OTOÑO
sombras_oto_prim = pd.concat([sombras_oto_prim, ad2], axis = 0)
sombras_oto_prim.drop_duplicates(inplace = True)
sombras_oto_prim['ID'] = [e for e in range(len(sombras_oto_prim))]
sombras_oto_prim.set_index('ID', drop = True, inplace = True)

#INVIERNO
sombras_inv_prim = pd.concat([sombras_inv_prim, ad3], axis = 0)
sombras_inv_prim.drop_duplicates(inplace = True)
sombras_inv_prim['ID'] = [e for e in range(len(sombras_inv_prim))]
sombras_inv_prim.set_index('ID', drop = True, inplace = True)

#VERANO
sombras_ver_prim = pd.concat([sombras_ver_prim, ad4], axis = 0)
sombras_ver_prim.drop_duplicates(inplace = True)
sombras_ver_prim['ID'] = [e for e in range(len(sombras_ver_prim))]
sombras_ver_prim.set_index('ID', drop = True, inplace = True)

In [ ]:
#Pintalabios
def extraer(URL):
    driver = webdriver.Chrome(PATH)
    driver.get(URL)
    
    driver.execute_script('window.scrollTo(0, 200);')  # 200px de scroll
    
    filas = driver.find_elements(By.CLASS_NAME, 'df-card__content')
    link = driver.find_elements(By.CLASS_NAME, 'df-card__main')
    tmp = []
    links = []

    for e in filas:
        tmp.append(e.text.split('\n'))
    
    for e in link:
        links.append(e.get_attribute('href'))
    
    cols = ['NOMBRE', 'TIPO', 'PRECIO']
    
    total = pd.DataFrame(tmp, columns = cols)
    
    total['LINKS'] = links
     
    return total

In [ ]:
links_pintalabios_primor = ['https://www.primor.eu/#/dfclassic/query=pintalabios%20vivid%20&filter%5Bcategories%5D%5B0%5D=Pintalabios%20y%20maquillaje%20de%20labios&session_id=e68b0b5ac1828fdb430ff5b492adb6e5&query_name=match_and',\
                           'https://www.primor.eu/#/dfclassic/query=pintalabios%20neutro&filter%5Bcategories%5D%5B0%5D=Barras%20de%20labios%20lujo&filter%5Bcategories%5D%5B1%5D=Pintalabios%20y%20maquillaje%20de%20labios&session_id=e68b0b5ac1828fdb430ff5b492adb6e5&query_name=match_and',\
                           'https://www.primor.eu/#/dfclassic/query=pintalabios%20oscuro&filter%5Bcategories%5D%5B0%5D=Pintalabios%20y%20maquillaje%20de%20labios&session_id=e68b0b5ac1828fdb430ff5b492adb6e5&query_name=match_and',\
                           'https://www.primor.eu/#/dfclassic/query=pintalabios%20rosados&filter%5Bcategories%5D%5B0%5D=Pintalabios%20y%20maquillaje%20de%20labios&filter%5Bbrand%5D%5B0%5D=Revolution&filter%5Bbrand%5D%5B1%5D=NYX%20Professional%20Makeup&filter%5Bbrand%5D%5B2%5D=Krash%20Kosmetics&session_id=e68b0b5ac1828fdb430ff5b492adb6e5&query_name=match_and']

In [ ]:
productos1 = Parallel(n_jobs=4, verbose=True)(delayed(extraer)(e) for e in links_pintalabios_primor)

In [ ]:
#Añadimos más filas
add1 = extraer('https://www.primor.eu/#/dfclassic/query=pintalabios%20rosados&filter%5Bcategories%5D%5B0%5D=Pintalabios%20y%20maquillaje%20de%20labios&filter%5Bbrand%5D%5B0%5D=NYX%20Professional%20Makeup&filter%5Bbrand%5D%5B1%5D=Krash%20Kosmetics&session_id=e68b0b5ac1828fdb430ff5b492adb6e5&query_name=match_and')
add2 = extraer('https://www.primor.eu/#/dfclassic/query=berry%20nars&filter%5Bbrand%5D%5B0%5D=NARS&filter%5Bcategories%5D%5B0%5D=Labios%20lujo&session_id=e68b0b5ac1828fdb430ff5b492adb6e5&query_name=match_or')
add3 = extraer('https://www.primor.eu/#/dfclassic/query=nude%20nars&session_id=e68b0b5ac1828fdb430ff5b492adb6e5&query_name=match_and')
add4 = extraer('https://www.primor.eu/#/dfclassic/query=dark%20nars&filter%5Bbrand%5D%5B0%5D=NARS&filter%5Bcategories%5D%5B0%5D=Barras%20de%20labios%20lujo&session_id=e68b0b5ac1828fdb430ff5b492adb6e5&query_name=match_or')
add5 = extraer('https://www.primor.eu/#/dfclassic/query=mac%20lips&session_id=e68b0b5ac1828fdb430ff5b492adb6e5&query_name=match_and')

In [ ]:
pintalab_ver_prim = productos1[0] 
pintalab_oto_prim = productos1[1] 
pintalab_inv_prim = productos1[2] 
pintalab_pri_pri = productos1[3]

In [ ]:
#Limpiamos los índices y añadimos lo que hemos sacado más

#PRIMAVERA
pintalab_pri_prim = pd.concat([pintalab_pri_pri, add, add3], axis = 0)
pintalab_pri_prim.drop_duplicates(inplace = True)
pintalab_pri_prim['ID'] = [e for e in range(len(pintalab_pri_prim))]
pintalab_pri_prim.set_index('ID', drop = True, inplace = True)

#OTOÑO
pintalab_oto_prim = pd.concat([pintalab_pri_pri, add1], axis = 0)
pintalab_oto_prim.drop_duplicates(inplace = True)
pintalab_oto_prim['ID'] = [e for e in range(len(pintalab_oto_prim))]
pintalab_oto_prim.set_index('ID', drop = True, inplace = True)

#INVIERNO
pintalab_inv_prim = pd.concat([pintalab_pri_pri, add4], axis = 0)
pintalab_inv_prim.drop_duplicates(inplace = True)
pintalab_inv_prim['ID'] = [e for e in range(len(pintalab_inv_prim))]
pintalab_inv_prim.set_index('ID', drop = True, inplace = True)

#VERANO
pintalab_ver_prim = pd.concat([pintalab_pri_pri, add5], axis = 0)
pintalab_ver_prim.drop_duplicates(inplace = True)
pintalab_ver_prim['ID'] = [e for e in range(len(pintalab_ver_prim))]
pintalab_ver_prim.set_index('ID', drop = True, inplace = True)

In [ ]:
#Colorete
def extraer(URL):
    driver = webdriver.Chrome(PATH)
    driver.get(URL)
    
    driver.execute_script('window.scrollTo(0, 200);')  # 200px de scroll
    
    filas = driver.find_elements(By.CLASS_NAME, 'df-card__content')
    link = driver.find_elements(By.CLASS_NAME, 'df-card__main')
    tmp = []
    links = []

    for e in filas:
        tmp.append(e.text.split('\n'))
    
    for e in link:
        links.append(e.get_attribute('href'))
    
    cols = ['NOMBRE', 'TIPO', 'PRECIO']
    
    total = pd.DataFrame(tmp, columns = cols)
    
    total['LINKS'] = links
     
    return total

In [ ]:
links_blush_primor = ['https://www.primor.eu/3ina/66421-colorete-the-blush.html#/dfclassic/query=blush&filter%5Bbrand%5D%5B0%5D=MAC&session_id=e68b0b5ac1828fdb430ff5b492adb6e5&query_name=match_and',\
                     'https://www.primor.eu/#/dfclassic/query=blush&filter%5Bbrand%5D%5B0%5D=NARS&session_id=e68b0b5ac1828fdb430ff5b492adb6e5&query_name=match_and',\
                     'https://www.primor.eu/#/dfclassic/query=blush%20rosa&filter%5Bbrand%5D%5B0%5D=3ina&session_id=e68b0b5ac1828fdb430ff5b492adb6e5&query_name=match_and',\
                     'https://www.primor.eu/#/dfclassic/query=blush&filter%5Bbrand%5D%5B0%5D=Yves%20Saint%20Laurent&session_id=e68b0b5ac1828fdb430ff5b492adb6e5&query_name=match_and']

In [ ]:
productos2 = Parallel(n_jobs=4, verbose=True)(delayed(extraer)(e) for e in links_blush_primor)

In [ ]:
blush_pri_prim = productos1[0] 
blush_ver_prim = productos1[1] 
blush_inv_prim = productos1[2] 
blush_oto_prim = productos1[3]

# **2 - EXTRACCIÓN DE DATOS DE PRIMOR DE SOMBRAS, PINTALABIOS Y COLORETE DE SEPHORA**

In [ ]:
PATH=ChromeDriverManager().install()

driver=webdriver.Chrome(PATH)  

In [ ]:
URL = 'https://www.urbandecay.com/eyeshadow/?prefn1=taxonomyColorEye&prefv1=BLACK%7CBLUE%7CBOLD%7CBROWN%7CCOOL%7CNEUTRALS%7CPLUMS%7CPURPLE%7CRED%7CSIENNA%7CSILVER%7CSMOKEY%7CTAUPE&start=0&sz=12#view-mode=grid'
driver.get(URL)

In [ ]:
prueba = driver.find_elements(By.CLASS_NAME, 'c-product-tile__caption-inner')

In [ ]:
driver.find_element(By.CLASS_NAME, 'c-product-tile__name').text

In [ ]:
driver.find_element(By.CLASS_NAME, 'c-product-tile__price').text

In [ ]:
textos = [e.text.split() for e in prueba]

In [ ]:
textos = textos[415::]

In [ ]:
driver.find_element(By.CSS_SELECTOR, 'body>div.c-modal.h-text-align-center.m-borderless.m-autosize-large.m-image.m-opened.h-fade-in>div>button') # acepta cookies


In [19]:
def extraer(URL):
    PATH=ChromeDriverManager().install()
    driver = webdriver.Chrome(PATH)
    driver.get(URL)
    
    time.sleep(5)
    driver.find_element(By.CSS_SELECTOR, 'body>div.c-modal.h-text-align-center.m-borderless.m-autosize-large.m-image.m-opened.h-fade-in>div>button').click() 
    time.sleep(5)
    driver.execute_script('window.scrollTo(0, 200);')  # 200px de scroll
    time.sleep(5)
    nombre = driver.find_elements(By.CLASS_NAME, 'c-product-tile__name')
    time.sleep(5)
    precio = driver.find_elements(By.CLASS_NAME, 'c-product-tile__price')
    tmp = []
    precios = []

    for e in nombre:
        tmp.append(e.text)
    
    for e in precio:
        precios.append(e.text.split())
    
    cols = ['NOMBRE']
    
    total = pd.DataFrame(tmp, columns = cols)
    
    total['PRECIO'] = precios
     
    return total

In [20]:
URL = 'https://www.urbandecay.com/eyeshadow/?prefn1=taxonomyColorEye&prefv1=BLACK%7CBLUE%7CBOLD%7CBROWN%7CCOOL%7CNEUTRALS%7CPLUMS%7CPURPLE%7CRED%7CSIENNA%7CSILVER%7CSMOKEY%7CTAUPE&start=0&sz=12#view-mode=grid'


In [18]:
extraer(URL)

2022-12-09 23:05:26,149 [39284] WARNING  py.warnings:109: [JupyterRequire] /var/folders/fg/hm_rz3k53fv5qf5dywc0bfwr0000gn/T/ipykernel_39284/2074955258.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)



IndexError: list index out of range